In [1]:
import os
import sys

cur_dir = os.getcwd()
pkg_rootdir = os.path.dirname(os.path.dirname(cur_dir))
# print(pkg_rootdir)
if pkg_rootdir not in sys.path:
    sys.path.append(pkg_rootdir)
# print(sys.path)

In [2]:
from Utils.mol2fp import morgan_binary_features_generator
fp_generator = morgan_binary_features_generator

In [3]:
import pandas as pd
train=pd.read_excel("../../Dataset/train_data.xlsx",index_col=False)
test=pd.read_excel("../../Dataset/test_data.xlsx",index_col=False)
out_test=pd.read_excel("../../Dataset/data_out_feats.xlsx",index_col=False)
X_train = [fp_generator(smi) for smi in train["smiles"]]
X_test = [fp_generator(smi) for smi in test["smiles"]]
X_out = [fp_generator(smi) for smi in out_test["smiles"]]

y_train_D=train['D']
y_train_P=train['P']
y_train_H=train['H']

y_test_D=test['D']
y_test_P=test['P']
y_test_H=test['H']

y_out_D=out_test['D']
y_out_P=out_test['P']
y_out_H=out_test['H']

[15:47:20] WARNING: not removing hydrogen atom without neighbors
[15:47:21] WARNING: not removing hydrogen atom without neighbors


In [4]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor
import numpy as np
import json

# Define the objective function for hyperparameter optimization
def objective(params, X_train, y_train):
  model = ExtraTreesRegressor(**params, random_state=42)
  # Perform cross-validation and minimize the negative mean squared error
  score = -np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
  return {'loss': score, 'status': STATUS_OK}

# Define the search space for hyperparameters
space = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 200]),
    'max_depth': hp.choice('max_depth', [10, 20, 30]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'criterion': 'friedman_mse',
    'bootstrap':True,
    'oob_score': True
}

# Function to optimize and save best parameters
def optimize_and_save(X_train, y_train, output_path):
  trials = Trials()
  best_params = fmin(fn=lambda params: objective(params, X_train, y_train), 
             space=space, algo=tpe.suggest, max_evals=50, trials=trials)
  
  # Convert categorical parameters back to their original values
  best_params['n_estimators'] = [50, 100, 200][best_params['n_estimators']]
  best_params['max_depth'] = [10, 20, 30][best_params['max_depth']]
  best_params['min_samples_split'] = [2, 5, 10][best_params['min_samples_split']]
  best_params['min_samples_leaf'] = [1, 2, 4][best_params['min_samples_leaf']]
  best_params['criterion'] = 'friedman_mse'
  best_params['bootstrap'] = True
  best_params['oob_score'] = True
  
  with open(output_path, 'w') as f:
    json.dump(best_params, f)
  
  return best_params

In [5]:
# Optimize and save parameters for D, P, and H
best_params_D = optimize_and_save(X_train, y_train_D, './results/morgan/best_params_D.json')
best_params_P = optimize_and_save(X_train, y_train_P, './results/morgan/best_params_P.json')
best_params_H = optimize_and_save(X_train, y_train_H, './results/morgan/best_params_H.json')

100%|██████████| 50/50 [19:25<00:00, 23.32s/trial, best loss: 11.0165441355612]  


In [6]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Function to train model, get feature importance, and evaluate performance
def train_and_evaluate(X_train, y_train, X_test, y_test,X_out,y_out, best_params):
  # Train model with best hyperparameters
  model = ExtraTreesRegressor(**best_params, random_state=42)
  model.fit(X_train, y_train)
  
  # Evaluate model
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)
  y_out_pred = model.predict(X_out)
  
  metrics = {
    'train_r2': r2_score(y_train, y_train_pred),
    'test_r2': r2_score(y_test, y_test_pred),
    'out_r2': r2_score(y_out, y_out_pred),
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
    'out_rmse': np.sqrt(mean_squared_error(y_out, y_out_pred)),
    'train_mae': mean_absolute_error(y_train, y_train_pred),
    'test_mae': mean_absolute_error(y_test, y_test_pred),
    'out_mae': mean_absolute_error(y_out, y_out_pred),
  
  }
  
  # Save original and predicted data
  train_results = pd.DataFrame({'y_train': y_train, 'y_train_pred': y_train_pred})
  test_results = pd.DataFrame({'y_test': y_test, 'y_test_pred': y_test_pred})
  out_results = pd.DataFrame({'y_out': y_out, 'y_out_pred': y_out_pred})

  
  return model,metrics,train_results, test_results,out_results

In [7]:
# Train and evaluate model
model_D,metrics_D,train_results_D, test_results_D,out_results_D = train_and_evaluate(X_train, y_train_D, X_test, y_test_D,X_out,y_out_D, best_params_D)
metrics_D

{'train_r2': 0.9618956202245981,
 'test_r2': 0.7814226146505492,
 'out_r2': 0.6983169802898215,
 'train_rmse': 0.3629927040758541,
 'test_rmse': 0.8359207377077681,
 'out_rmse': 0.765708242701564,
 'train_mae': 0.2537196635670927,
 'test_mae': 0.5601506487710544,
 'out_mae': 0.588664702880925}

In [8]:
# Train and evaluate model
model_P,metrics_P,train_results_P, test_results_P,out_results_P = train_and_evaluate(X_train, y_train_P, X_test, y_test_P,X_out,y_out_P, best_params_P)
metrics_P

{'train_r2': 0.9126146095832235,
 'test_r2': 0.5545721998251523,
 'out_r2': 0.47841915030192006,
 'train_rmse': 1.2817684663465392,
 'test_rmse': 2.7718883775168615,
 'out_rmse': 3.823311795753223,
 'train_mae': 0.9152498193507754,
 'test_mae': 1.923162103937849,
 'out_mae': 2.592588624028432}

In [9]:
# Train and evaluate model
model_H,metrics_H,train_results_H, test_results_H,out_results_H = train_and_evaluate(X_train, y_train_H, X_test, y_test_H,X_out,y_out_H, best_params_H)
metrics_H

{'train_r2': 0.947000358125516,
 'test_r2': 0.7473570104775835,
 'out_r2': 0.7367782433563308,
 'train_rmse': 1.3404800015106961,
 'test_rmse': 2.2371062872174123,
 'out_rmse': 3.73824906314914,
 'train_mae': 0.7953238334846969,
 'test_mae': 1.5599048563357916,
 'out_mae': 2.684249957494356}

In [10]:
# 批量保存 metrics 到 Excel 文件
metrics_dict = {
  "Metrics_D": metrics_D,
  "Metrics_P": metrics_P,
  "Metrics_H": metrics_H
}

output_path = "./results/morgan/metrics_summary.xlsx"

with pd.ExcelWriter(output_path) as writer:
  for sheet_name, metrics in metrics_dict.items():
    pd.DataFrame([metrics]).to_excel(writer, sheet_name=sheet_name, index=False)

In [11]:
output_path = "./results/morgan/train_results_summary.xlsx"

with pd.ExcelWriter(output_path) as writer:
  train_results_D.to_excel(writer, sheet_name="Train_Results_D", index=False)
  train_results_P.to_excel(writer, sheet_name="Train_Results_P", index=False)
  train_results_H.to_excel(writer, sheet_name="Train_Results_H", index=False)

In [12]:
output_path = "./results/morgan/test_results_summary.xlsx"

with pd.ExcelWriter(output_path) as writer:
  test_results_D.to_excel(writer, sheet_name="Test_Results_D", index=False)
  test_results_P.to_excel(writer, sheet_name="Test_Results_P", index=False)
  test_results_H.to_excel(writer, sheet_name="Test_Results_H", index=False)

In [13]:
output_path = "./results/morgan/out_results_summary.xlsx"

with pd.ExcelWriter(output_path) as writer:
  out_results_D.to_excel(writer, sheet_name="out_Results_D", index=False)
  out_results_P.to_excel(writer, sheet_name="out_Results_P", index=False)
  out_results_H.to_excel(writer, sheet_name="out_Results_H", index=False)